In [1]:
### import necessary libraries
import ee
import json
import geemap
import sklearn
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

ee.Authenticate()
ee.Initialize(project="ee-franciscofurey")

In [13]:
dataset = ee.ImageCollection("ESA/WorldCover/v100").first()

# Select a Basin
basin = ee.FeatureCollection("WWF/HydroSHEDS/v1/Basins/hybas_7")
selected = basin.filter(ee.Filter.eq('HYBAS_ID', 4071139640))
geometry = selected.geometry()

Map = geemap.Map()

Map.centerObject(geometry)
# Add the classified image
classification = dataset.select('Map').clip(geometry)

Map.addLayer(classification, {}, 'WorldCover Classification')

# Let's calculate area
# Vector Area Calculation
# geometry -> Area()
basin_area = geometry.area().divide(1e6)
print('Basin Area:', basin_area.getInfo())

# Compare area Calculations from other software you must specify a projection

basin_area = geometry.area(
    maxError=0.1, 
    proj='EPSG:32643'
    ).divide(1e6)
print('Basin Area EPSG:32643', basin_area.getInfo())

water = classification.eq(80)

# Calculate the area of water
# Lets compute area of each pixel
water_area = water.multiply(ee.Image.pixelArea()).divide(1e6)

# Sum the area of all the pixels

water_area = water_area.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=geometry,
    scale=10,
    maxPixels=1e10,
    tileScale=16
    )

print('Water Area:', water_area.getInfo())

Basin Area: 4178.032796336794
Basin Area EPSG:32643 4166.064852187439
Water Area: {'Map': 19.114295885594963}
